# <span style='font-family: CMU Sans Serif, sans-serif;'> Gradient based learning  </span> 

By using non-linear activation functions the causes most loss-functions to be non-convex which implies that we cannot solve it analytically. We have to ressource to using iterative, gradient-based optimizers that drive the loss function toa low value (i.e., we cannot solve for the weights minimizing loss, but we have to numerically approximate them).

The training algorithm for neural networks almost always uses the gradient of the loss function to descent it. Specific ways of doing so is discussed later (e.g., SGD, mini-batch-GD, GD, etc.). The specific algorithms are improvements and refinements on the generel idea of gradient descent. For neural networks we can still compute the gradient of the loss-function efficiently and exactly (however, it is more difficult). To use gradient descent we must define a cost function. 

## <span style='font-family: CMU Sans Serif, sans-serif;'> Cost functions  </span> 

Important to choose good cost function when creating a neural network. Cost function for neural nets are large the same as for parametric models.

Parametric models define a distribution $p(\mathbf{y} | \mathbf{x}; \mathbf{\theta})$ and we use maximum likelihood. This means we use cross-entropy between training data and the models predictions as the cost function; how likely is the observed data given the model parameters.  

A simpler approach: instead of predicting a complete probability distribution over $\mathbf{y}$, we predict some statistic of $\mathbf{y}$ conditioned on $\mathbf{x}$. Specialized loss functions enable us ot train a predictor of these estimates.

The total cost function usually includes the primary cost functions just described and a regularization term. A popular choice for both parametric and neural network models is the **weight decay** regularization, which is a criterion that expresses a preference for the weights to have smaller squared $L^{2}$ norm ($\lambda \mathbf{w}^{\top}\mathbf{w}$)

### <span style='font-family: CMU Sans Serif, sans-serif;'> Learning conditional distributions with maximum likelihood  </span> 

<div style='border: 1px solid currentColor; border-radius: 2px; padding: 10px; width: 95%;'>

<span style='font-family: CMU Sans Serif, sans-serif; font-weight: bold; font-size: 17px;'>
Cost function derived from maximum likelihood
</span> 

Most NNs are trained using maximum likelihood. The cost function is simply the negative log-likelihood, also called the cross-entropy between the training data and the model distribution. This cost function is given by
\begin{equation}
    J(\mathbf{\theta}) = - \mathbb{E}_{\mathbf{x}, \mathbf{y} \sim \hat{p}_{\text{data}}} \log p_{\text{model}}(\mathbf{y} | \mathbf{x}) \tag{1}
\end{equation}

The specific form (1) changes depending on the specific form of $\log p_{\text{model}}$. An advantage of deriving the cost function from maximum likelihood is that we do not have to create cost functions for every type of model; specifying a model $p(\mathbf{y} | \mathbf{x})$ specifies a cost function.
</div>

The gradient of the cost function must be large and predictable enough to serve as a guide for the learning algorithm. Function that saturate undermine this objective since the gradient becomes small. Often this happens because activation function s used to produce the output of hidden units or the output is saturate. The negative log-likelihood helps alleviate this issue. Activation functions with $\exp$ can saturate when their arguments become very negative, and the $\log$ of the negative log-likelihood undoes this.


A usual property of the cross-entropy cost used to perform maximum likelihood estimation is that it usually does not have a minimum value when applied to these types of models. Also for discrete input values models are parametrized so they cannot represent a probability of zero or one, but rather come arbitrarily close. An example of this is logistic regression, where it can assign extremely high densities to the correct training set outputs, resulting in cross-entropy approaching negative infinity. We later discuss ways of modifying the learning problem so the model cannot reap unlimited reward this way.

### <span style='font-family: CMU Sans Serif, sans-serif;'> Learning conditional statistics  </span> 

Instead of learning $p(\mathbf{y} | \mathbf{x}; \boldsymbol{\theta} )$ we just want to learn a conditional statistic of $\mathbf{y}$ given $\mathbf{x}$. For example, if we have the predictor $f(\mathbf{x};\boldsymbol{\theta} )$ we might want to employ it to predict the mean of $\mathbf{y}$ given $\mathbf{x}$.

If we have sufficiently powerful neural networks we can think of the NN as being able to represent any function $f$, which this function only being limited by features such as continuity and boundedness rather than having a specific parametric form. From this point of view we can view the cost function as a **functional** rather than a function. A functional is a mapping from functions to real numbers. An example of a functional is the mean; for a continuous function $f(x)$ over an interval $[a,b]$ the mean is $M(f) = \frac{1}{b-a} \int_a^{b}f(x) \, dx$, and since it takes an entire function is input, we can consider it a functional (the same goes for discrete values, where the integral becomes a sum).

Learning then becomes choosing a function rather than a set of parameters. We can design the cost functional to have its minimum lie on the function that maps $\mathbf{x}$ to the expected value of $\mathbf{y}$ given $\mathbf{x}$.

Using calculus of variation we can derive that solving the optimization problem
$$
f^{*} = \operatorname{argmin}_f \mathbb{E}_{\mathbf{x}, \mathbf{y} \sim p_{\text{data}}} \left\| \mathbf{y} - f(\mathbf{x}) \right\|^{2}
$$
yields 
$$
f^{*}(\mathbf{x}) = \mathbb{E}_{\mathbf{y}\sim  p_{\text{data}}(\mathbf{y} | \mathbf{x})}[\mathbf{y}]
$$ 
so long as this functions lies within the class we optimize over. Thus, we could train on infinitly many samples from the true data generating distribution, minimizing the mean squared error cost function would give a function that predicts the mean of $\mathbf{y}$ given $\mathbf{x}$. 

We also find that solving this optimization function
$$
f^{*} = \operatorname{argmin}_f \mathbb{E}_{\mathbf{x}, \mathbf{y} \sim  p_{\text{data}}} \left\| \mathbf{y} - f(\mathbf{x}) \right\|_1
$$
yields a function that predicts the median value of $\mathbf{y}$ given $\mathbf{x}$. This cost function is called the **mean absolutte error**. 

However, the mean squared error and mean absolutte error often lead to poor results when used with gradient-based optimization. Some output units that saturate produce very small gradients when combined with these cost functions making training slow. Additionally, MSE is sensitive to outliers and MAE has issues with gradient calculations since it is not differentiable at zero. This is one reason that the cross-entropy cost function is more popular than meansquared error or mean absolute error, even when it is not necessary to estimate anentire distribution $p(\mathbf{y} | \mathbf{x})$.


#### <span style='font-family: CMU Sans Serif, sans-serif;'> Relating to stock analysis  </span> 

The noise in stock data is usually assumed to follow a normal (Gaussian) distribution in many theoretical models, particularly in the short term. This assumption is central to models like the Black-Scholes option pricing model and random walk theory.

However, empirical studies of financial markets have shown that stock returns often exhibit fat tails and leptokurtosis, meaning that extreme events (large price swings) occur more frequently than predicted by a normal distribution. Because of this, alternative distributions like the student’s t-distribution, Lévy distributions, or even power-law distributions are sometimes used to better capture the statistical properties of market noise.

If we assume that the noise in stock data follows a Gaussian distribution then using the mean squared error statistic for training becomes clear. If we have that 
$$
p(\mathbf{y}|\mathbf{x}; \boldsymbol{\theta}) = \frac{1}{\sqrt{2\pi\sigma^{2}}} \exp\left( - \frac{(y - f(\mathbf{x};\boldsymbol{\theta} ))^{2}}{2\sigma^{2}} \right) 
$$
, then the log-likelihood for a single datapoint $(\mathbf{x}, \mathbf{y})$ is
$$
\log p(\mathbf{y}|\mathbf{x};\boldsymbol{\theta}) = - \frac{\left\| \mathbf{y} - f(\mathbf{x}; \boldsymbol{\theta} ) \right\|^{2}}{2\sigma^{2}} - \log\sqrt{2\pi\sigma^{2}}
$$

The negative log-likelihood is then 
$$
\mathcal{L}_{\text{NLL}}(\boldsymbol{\theta}) = \frac{\left\| \mathbf{y} - f(\mathbf{x};\boldsymbol{\theta} ) \right\|^{2}}{2\sigma^{2}} + \log \sqrt{2\pi\sigma^{2}}
$$

To train the NN we minimize the expected NLL over the data distribution $p_{\text{data}}(\mathbf{x},\mathbf{y})$ which is
$$
\mathcal{L}(\boldsymbol{\theta} ) = \mathbb{E}_{\mathbf{x},\mathbf{y} \sim p_{\text{data}}}\left[ \frac{\left\| \mathbf{y}-f(\mathbf{x};\boldsymbol{\theta} ) \right\|^{2}}{2\sigma^{2}} + \log\sqrt{2\pi\sigma^{2}} \right] 
$$
and since $\sqrt{2\pi\sigma^{2}}$ is a constant minimizing $\mathcal{L}(\boldsymbol{\theta} )$ is equivalent to minimizing 
$$
\mathbb{E}_{\mathbf{x},\mathbf{y} \sim p_{\text{data}}} \left[ \left\| \mathbf{y}-f(\mathbf{x};\boldsymbol{\theta} ) \right\|^{2} \right] 
$$
which is the mean squared error. **Thus, minimizing the negative log-likelihood under an assumption that stock noise is Gaussian is equivalent to minimizing the mean squared error loss**. Using the mean squared error as loss finds the function $f(\mathbf{x})$ that minimizes the expected squared error between predictions and labels. this means that $f^{*}(\mathbf{x})$ is the conditional expectation of $\mathbf{y}$ given $\mathbf{x}$.

All this assumes fixed variance. We can also assume that $\sigma^{2}(\mathbf{x})$, which would yield heteroscedastic regression (i.e., the variance can change over time). A potentially interesting subject, but for now beyond our scope.

## <span style='font-family: CMU Sans Serif, sans-serif;'> Output units  </span> 

Often we the cross-entropy and the choice of how to represent the output then determines the form of cross-entropy function.

The following functions can in principle be used on the output of the hidden layers, however the focus here is in the final/output layer.

Suppose the feedforward net provides a set of hidden features defined by $\mathbf{h} = f(x;\boldsymbol{\theta} )$. The output layer then provides some additional transformation from the features to complete the task that the network must perform.

### <span style='font-family: CMU Sans Serif, sans-serif;'> Linear units for Gaussian output distributions  </span> 

One simple kind of output unit is based on an affine transformation with n ononlinearity. These are often just called linear units.

Given features $\mathbf{h}$, a layer of linear output units produces a vector $\hat{\mathbf{y}} = \mathbf{W}^{\top}\mathbf{h} + \mathbf{b}$. 

Linear units are often used to produce the mean of a conditional Gaussian distribution
$$
p(\mathbf{y}|\mathbf{x}) = \mathcal{N}(\mathbf{y}; \hat{\mathbf{y}}, \mathbf{I})
$$

Maximizing the log-likelihood is equivalent to minimizing the mean squared error (as stated earlier). Because linear units do not saturate, they pose little difficulty for gradient-based optimization algorithms and may be used with a wide variety of optimization algorithms.

### <span style='font-family: CMU Sans Serif, sans-serif;'> Sigmoid units for Bernoulli output distributions </span> 

Used when dealing with binary classification of a variable $y$. 

The maximum likelihood approach is to define a Bernoulli distribution over $\mathbf{y}$ conditioned on $\mathbf{x}$. 

A Bernoulli distribution is defined by a single number which the NN must predict, $P(y=1 | \mathbf{x}) \in [0,1]$. Satisfying this constraint is not simple. Suppose we use a linear unit and threshold its value to obtain a valid probability:
$$
P(y = 1 | \mathbf{x}) = \max\{ 0, \min\{ 1, \mathbf{w}^{\top}\mathbf{h} + b \} \}. \tag{2}
$$
This yields a valid conditional distribution, but it is difficult to train with gradient descent. When $\mathbf{w}^{\top} \mathbf{h} + b < 0$ the gradient of the output with respect to its parameters would be $\boldsymbol{0}$; thus, the learning algorithm does not have a guide for how to improve the corresponding parameters.

We want an activation function that yields a strong gradient when the model is incorrect. We use the sigmoid output units combined with maximum likelihood.

A sigmoid output unit is given by
$$
\hat{y} = \sigma \left( \mathbf{w}^{\top}\mathbf{h} + b \right) , \tag{3}
$$
where $\sigma$ is the loistic sigmoid function. The sigmoid output unit has two components: (1) it uses a linear layer to compute $z = \mathbf{w}^{\top}\mathbf{h} + b$, (2) uses the sigmoid activation function to convert $z$ into a probability.

The sigmoid can be motivated by constructing an unnormalized probability distribution $\tilde{P}(y)$, which does not sum to $1$. We divide by an appropriate constant to obtain a valid probability distribution. If we start by assuming that the unnormalized $\log$-probabilities are linear in $y$ and $z$, we can exponentiate to obtain the unnormalized probabilities.  We then normalize to see that this yields a Bernoulli distribution controlled by a sigmoidal transformation of $z$ 
\begin{align*}
    \log \tilde{P}(y) &= yz \\
    \tilde{P}(y) &= \exp(yz) \\
    P(y) &= \frac{\exp(yz)}{\sum_{y'=0}^{1}\exp(y'z)} \\
    P(y) &= \sigma ((2y - 1)z)
\end{align*}
    
The $z$ variable defining a distribution based on exponentiation and normalization are common over binary variables is called a **logit**.


It is natural to use this approach with maximum likelihood learning since the $\log$ undoes the exponentiation and hereby addresses the saturation of the sigmoid function which improves gradient descent based training.


The loss function for maximum likelihoods learning of a Bernoulli parametrized by a sigmoid is
\begin{align*}
    J(\boldsymbol{\theta}) &= - \log P(y | \mathbf{x}) \\
    &= -\log \sigma((2y - 1)z) \\
    &= \zeta((1 - 2y)z)
\end{align*}

where $\zeta(x) = \log(1+\exp(x))$ is called the softplus function. By rewriting in terms of the softplus function we see that saturation only becomes a problem at very negative values of  $(1-2y)z$. Saturation only occurs when the model already has the right answer and in that case the parameters need not to be updated. When $z$ has the wrong sign, $(1-2y)z$, simplifies to $\vert z \vert$, and the softplus function asymptotes to returning $\vert z \vert$, whilst the derivative with respect to $z$ asymptotes to $\operatorname{sign}(z)$, so in the limit of very incorrect $z$ the softplus function does not shrink the gradient at all. This means that gradient-based learning can quickly correct a mistaken $z$. 

Analytically, the logarithm of the sigmoid is always defined and finite, because the sigmoid returns values restricted to the open interval $(0,1)$, rather than using the entire closed interval of valid probabilities $[0,1]$.

### <span style='font-family: CMU Sans Serif, sans-serif;'> Softmax units for Multinoulli output distributions   </span> 

Used when we want to represent a probability distributed over a discrete variable with $n$ possible values, we may use the $\operatorname{softmax}$ function. $\operatorname{softmax}$ is used as the output of a classifier to represent probabilities over $n$ classes, but it can rarely be used inside when we want the model to choose between one of $n$ possible options for some internal variable.

We need a vector $\hat{\mathbf{y}}$ with $\hat{y}_i = P(y=i | \mathbf{x})$ where $\hat{y}_i \in [0,1] \, \forall i \in n$ and $\sum_{i \in [n]} \hat{y}_i = 1$.  The same approach that worked forthe Bernoulli distribution generalizes to the multinulli distribution. First, a linear layer predicts unnormalized log probabilities
$$
\mathbf{z} = \mathbf{W}^{\top}\mathbf{h} + \mathbf{b}
$$
where $z_i = \log \tilde{P}(y = i | \mathbf{x})$ (the reason why the unnormalized log probabilities can take on values $>1$ is because the are unnormalized). The $\operatorname{softmax}$ function can then exponentiate and normalize $\mathbf{z}$ to obtain the desired $\hat{\mathbf{y}}$. Formally the $\operatorname{softmax}$ is given by
$$
\operatorname{softmax}(\mathbf{z})_i = \frac{\exp(z_i)}{\sum_j \exp(z_j)}
$$

As with the logistic sigmoid, the use of theexpfunction works well whentraining the softmax to output a target valueyusing maximum log-likelihood. Here we want to maximize $P(y=i; \mathbf{z}) = \log \operatorname{softmax}(\mathbf{z})_i$.  Defining the $\operatorname{softmax}$ in terms of $\exp$ is natural because thelogin the log-likelihood can undo the exp of the $\operatorname{softmax}$
$$
\log \operatorname{softmax}(\mathbf{z})_i = z_i - \log \sum_j \exp(z_j) \tag{4}
$$
From $(4)$ we see that $z_i$ always has a direct contribution to the cost and since this terms cannot saturate training can always proceed (even when the second term in $(4)$ is small).


Many objective functions other than the log-likelihood do not work as well with the softmax function. Specifically, objective functions that do not use a $\log$ to undo the $\exp$ of the $\operatorname{softmax}$ fail to learn when the argument to the $\exp$ becomes very negative, causing the gradient to vanish. In particular, squared error is a poor loss function for $\operatorname{softmax}$ units and can fail to train the model to change its output, even when the model makes highly confident incorrect predictions. 

The $\operatorname{softmax}$ can saturate. The outputs can saturate when the differences between input becomes extreme. When the $\operatorname{softmax}$ saturates many cost function based on the $\operatorname{softmax}$ also saturates, unless they invert the saturating activating function. To see this first observe that 
$$
\operatorname{softmax}(\mathbf{z}) = \operatorname{softmax}(\mathbf{z} + c)
$$
which we use to derive a numerically stable variant of the $\operatorname{softmax}$
$$
\operatorname{softmax}(\mathbf{z}) = \operatorname{softmax}(\mathbf{z} - \max_i z_i)
$$
which allows us to evaluate $\operatorname{softmax}$ with only small numerical errors, and here we see that the $\operatorname{softmax}$ is driven by the amount its arguments deviate from $\max_i z_i$. We see that $\operatorname{softmax}(\mathbf{z})$ saturates to $1$ when $z_i = \max_i z_i$ and all other values are far smaller than $\max_i z_i$, and to $0$ reverse when $z_i \neq \max_i z_i$ and other $z_i$ are far smaller. This can cause learning issues.

$\mathbf{z}$ can be produces two different ways. 
The most common is with $\mathbf{z} = \mathbf{W}^{\top}\mathbf{h} + \mathbf{b}$. This overparametrizes the distribution. The constraint $\sum_{i \in [n]}$ means we only need $n-1$ to estimate the distribution. We can thus impose a requirement that one element of $\mathbf{z}$ must be fixed. For example we can impose that $z_n = 0$ which is what the sigmoid does when defining $P(y=1 | \mathbf{x}) = \sigma(z)$ which is equivalent to defining $P(y=1 | \mathbf{x}) = \operatorname{softmax}(\mathbf{z})_1$ with a two-dimensional $\mathbf{z}$ and $z_1 = 0$. These two methods are the same, and there is often no difference between the overparametrized and restricted version.

### <span style='font-family: CMU Sans Serif, sans-serif;'> Other output types  </span> 

Neural networks can generalize to almost any kind of output layer that we wish. The principle of maximum likelihood provides a guide for how to design a good cost function for nearly any kind of output layer.

#### <span style='font-family: CMU Sans Serif, sans-serif;'> Heteroscedastic models  </span> 

In general, if we define a conditional distribution $p(\mathbf{y} | \mathbf{x}; \boldsymbol{\theta} )$ the principle of maximum likelihood suggests we use $- \log p(\mathbf{y} | \mathbf{x}; \boldsymbol{\theta} )$ as our cost function.
Also, in general, we think of a NN as representing a function $f(\mathbf{x}; \boldsymbol{\theta} )$. The outputs of this function are not directly predictions of $\mathbf{y}$. Instead, $f(\mathbf{x}; \boldsymbol{\theta} ) = \boldsymbol{\omega}$ provides the parameteres for a distribution over $y$. Our loss can then be interpreted as $- \log p(\mathbf{y}; \boldsymbol{\omega} (\mathbf{x})$.

We can use this to learn the variance of a conditional Gaussian for $\mathbf{y}$, given $\mathbf{x}$. We can include the variance in as one of the properties of the distribution $p(\mathbf{y} | \mathbf{x})$ that is controlled by $\boldsymbol{\omega} = f(\mathbf{x}; \boldsymbol{\theta} )$. The negative log-likelihood $- \log p(\mathbf{y}; \boldsymbol{\omega} (\mathbf{x}))$ then provides a cost function with the appropriate terms necessary to make our optimization procedure incrementally learn the variance. We can then model the amount of variance in $\mathbf{y}$ given $\mathbf{x}$. This is called a heteroscedastic model, where we make the specification of the variance being one fo the values output by $f(\mathbf{x}; \boldsymbol{\theta} )$. This is typically done by formulating the Gaussian using precision rather than variance.

#### <span style='font-family: CMU Sans Serif, sans-serif;'> Mixture density networks  </span> 

We often want to perform multimodal regression, where we predict real values from a conditional distribution $p(\mathbf{y} | \mathbf{x})$ that can have several different peaks in $\mathbf{y}$ space for the same value of $\mathbf{x}$. IN this case, a Gaussian mixture is a good representation of the output. NNs with Gaussian mixtures as output are called **mixture density networks**. 

A Gaussian mixture output with $n$ components is defined by the conditional probability distribution
$$
p(\mathbf{y} | \mathbf{x}) = \sum_{i=1}^{n} p(c = i | \mathbf{x}) \mathcal{N}(\mathbf{y}; \boldsymbol{\mu}^{(i)}(\mathbf{x}), \boldsymbol{\Sigma} ^{(i)}(\mathbf{x}) )
$$ 

The neural network then has three different outputs: a vector defining $p(c = i | \mathbf{x})$, a matrix providing $\boldsymbol{\mu} ^{(i)}(\mathbf{x})$ for all $i$, and a tensor providing $\boldsymbol{\Sigma} ^{(i)}(\mathbf{x})$ for all $i$. These outputs must satisfy the following constraints:
1. Mixture components $p(c=i | \mathbf{x})$: these form a multinoulli distribution over the $n$ different components associated with latent variable $c$, and can typically be obtained by a $\operatorname{softmax}$ over an $n$ -dimensional vector, to guarantee that these outputs are positive and sum to $1$.
2. Mean $\boldsymbol{\mu} ^{(i)}(\mathbf{x})$: these indicate the mean associated with the $i$-th Gaussian component and are unconstrained (typically no nonlinearity at all for these output units). If $\mathbf{y}$ is a $d$-vector then the network must output an $n \times d$ matrix containing all $n$ of these $d$-dimensional vectors. Leaning these means with maximum likelihood is more complicated than learning the means of a distribution with only one output mode. We only want to update the mean for the component that actually produced the observation. In practice, we do not know which component produced each observation. The expression for the negative lig-likelihood weights each example's contribution to the loss for each component by the probability that the component produced the example.
3. Covariances $\boldsymbol{\Sigma} ^{(i)}(\mathbf{x})$: these specify the covariance matrix for each component $i$. as when learning as single Gaussian component, we typically use a diagonal matrix to avoid needing to compute determinants. As with learning the means of the mixture, maximum likelihood is complicated by needing to assign partial responsibility for each point to each mixture component. Gradient descent will automatically follow the correct process if given the correct specification of the negative log-likelihood under the mixture model.

Some reports show that gradient-based optimization of conditional Gaussian mixtures can be unreliable, in part since one gets divisions by the variance which can be numerically unstable (e.g. when some variances get small). One solution is to **clip gradients** while another is to **scale the gradients heuristically**.